# Explore DWD Data API

```shell
pip install dwdweather2
pip install pandas
```


In [ ]:
from datetime import datetime, timedelta
from dwdweather import DwdWeather
import pandas as pd
import numpy as np

In [ ]:
# Make DwdWeather instances
dwd_60m = DwdWeather(resolution="hourly")
dwd_10m = DwdWeather(resolution="10_minutes")

In [ ]:
# latitude and longitude Ulm; surrounding stations
params = dict()

In [ ]:
class StationList(DwdWeather):
    def __init__(self, resolution="hourly", lon=7.0, lat=51.0, surrounding=15000):
        super().__init__(resolution)
        stations = self.nearest_station(lon=lon, lat=lat, surrounding=surrounding)
        self.df = pd.DataFrame(stations)

    def get_active_stations(self):
        days_since_active = datetime.now() - pd.to_datetime(self.df.date_end, format="%Y%m%d")
        bool_index = days_since_active < timedelta(days=7)
        active_index = np.where(bool_index)[0]
        return self.df.iloc[active_index]

    def get_wind_stations(self, actives_only=True, sample_delay=7):
        if actives_only:
            df = self.get_active_stations()
        else:
            df = self.df

        sample_timestamp = datetime.now() - timedelta(days=sample_delay)
        has_winddir = {}
        has_windspd = {}

        for i in range(len(df)):
            station_id = df.iloc[i].station_id
            sample = self.query(station_id=station_id, timestamp=sample_timestamp)
            if sample:
                if sample["wind_speed"]:
                    has_windspd[i] = True
                if sample["wind_direction"]:
                    has_winddir[i] = True

        return pd.DataFrame([has_windspd, has_winddir])

In [ ]:
stl = StationList(resolution="10_minutes", lon=9.98, lat=48.40)
stl.get_active_stations()

In [ ]:
dwd_10m.query(station_id=15444, timestamp=datetime(2021,1,1,13))